In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests


from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
# -----------------------------------------------------------------------

pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

In [9]:
diccionario = {'Genero': ["Drama"],
                    'Tipo' : ["Movie"],
                    'Nombre' : ["The Hunger Games"],
                    'Año estreno' : [2012],
                    'Mes estreno' : [12],
                    'Id' :['tt0313462']}


driver = webdriver.Chrome()

driver.get("https://www.imdb.com/search/title/")

driver.maximize_window()

driver.find_element("css selector", "#main > div:nth-child(3) > div.inputs > input").send_keys(diccionario["Nombre"][0], Keys.ENTER)

